In [1]:
import numpy as np

In [11]:
def needleman_wunsch(seq1, seq2, match=2, mismatch=-1, gap_penalty=-4):
    """
    Алгоритм Нидлмана-Вунша
    """
    M = len(seq1)
    N = len(seq2)
    
    F = [[0] * (N + 1) for _ in range(M + 1)]
    Ptr = [[''] * (N + 1) for _ in range(M + 1)]

    for j in range(1, N + 1):
        F[0][j] = j * gap_penalty
        Ptr[0][j] = 'LEFT'
    
    for i in range(1, M + 1):
        F[i][0] = i * gap_penalty
        Ptr[i][0] = 'UP'

    for i in range(1, M + 1):
        for j in range(1, N + 1):

            if seq1[i-1] == seq2[j-1]:
                score = match
            else:
                score = mismatch

            diag = F[i-1][j-1] + score
            up = F[i-1][j] + gap_penalty
            left = F[i][j-1] + gap_penalty
            
            max_score = max(diag, up, left)
            F[i][j] = max_score
            
            if max_score == diag:
                Ptr[i][j] = 'DIAG'
            elif max_score == up:
                Ptr[i][j] = 'UP'
            else:
                Ptr[i][j] = 'LEFT'
    
    align1 = ""
    align2 = ""
    i, j = M, N
    
    while i > 0 or j > 0:
        if i > 0 and j > 0 and Ptr[i][j] == 'DIAG':
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
        elif i > 0 and Ptr[i][j] == 'UP':
            align1 = seq1[i-1] + align1
            align2 = '-' + align2
            i -= 1
        else:
            align1 = '-' + align1
            align2 = seq2[j-1] + align2
            j -= 1
    
    return F[M][N], align1, align2, F, Ptr

In [18]:
def needleman_wunsch_affine(seq1, seq2, match=2, mismatch=-1, open_gap=-10, extend_gap=-1):
    M, N = len(seq1), len(seq2)
    NEG_INF = -10**9
    
    M_mat = [[NEG_INF] * (N + 1) for _ in range(M + 1)]
    I1 = [[NEG_INF] * (N + 1) for _ in range(M + 1)]
    I2 = [[NEG_INF] * (N + 1) for _ in range(M + 1)]
    
    Ptr_M = [[''] * (N + 1) for _ in range(M + 1)]
    Ptr_I1 = [[''] * (N + 1) for _ in range(M + 1)]
    Ptr_I2 = [[''] * (N + 1) for _ in range(M + 1)]
    
    M_mat[0][0] = 0
    
    for j in range(1, N + 1):
        I2[0][j] = open_gap + (j - 1) * extend_gap
        Ptr_I2[0][j] = 'I2'
    
    for i in range(1, M + 1):
        I1[i][0] = open_gap + (i - 1) * extend_gap
        Ptr_I1[i][0] = 'I1'
    
    for i in range(1, M + 1):
        for j in range(1, N + 1):
            s = match if seq1[i-1] == seq2[j-1] else mismatch
            
            candidates_M = [M_mat[i-1][j-1] + s, I1[i-1][j-1] + s, I2[i-1][j-1] + s]
            M_mat[i][j] = max(candidates_M)
            if M_mat[i][j] == candidates_M[0]:
                Ptr_M[i][j] = 'M'
            elif M_mat[i][j] == candidates_M[1]:
                Ptr_M[i][j] = 'I1'
            else:
                Ptr_M[i][j] = 'I2'
            
            candidates_I1 = [M_mat[i-1][j] + open_gap, I1[i-1][j] + extend_gap]
            I1[i][j] = max(candidates_I1)
            if I1[i][j] == candidates_I1[0]:
                Ptr_I1[i][j] = 'M'
            else:
                Ptr_I1[i][j] = 'I1'
            
            candidates_I2 = [M_mat[i][j-1] + open_gap, I2[i][j-1] + extend_gap]
            I2[i][j] = max(candidates_I2)
            if I2[i][j] == candidates_I2[0]:
                Ptr_I2[i][j] = 'M'
            else:
                Ptr_I2[i][j] = 'I2'
    
    final_score = max(M_mat[M][N], I1[M][N], I2[M][N])
    
    if final_score == M_mat[M][N]:
        state, i, j = 'M', M, N
    elif final_score == I1[M][N]:
        state, i, j = 'I1', M, N
    else:
        state, i, j = 'I2', M, N
    
    align1, align2 = "", ""
    
    while i > 0 or j > 0:
        if state == 'M':
            if Ptr_M[i][j] == 'M':
                align1 = seq1[i-1] + align1
                align2 = seq2[j-1] + align2
                i -= 1
                j -= 1
                state = 'M'
            elif Ptr_M[i][j] == 'I1':
                align1 = seq1[i-1] + align1
                align2 = seq2[j-1] + align2
                i -= 1
                j -= 1
                state = 'I1'
            else:
                align1 = seq1[i-1] + align1
                align2 = seq2[j-1] + align2
                i -= 1
                j -= 1
                state = 'I2'
        
        elif state == 'I1':
            if Ptr_I1[i][j] == 'M':
                align1 = seq1[i-1] + align1
                align2 = '-' + align2
                i -= 1
                state = 'M'
            else:
                align1 = seq1[i-1] + align1
                align2 = '-' + align2
                i -= 1
                state = 'I1'
        
        else:
            if Ptr_I2[i][j] == 'M':
                align1 = '-' + align1
                align2 = seq2[j-1] + align2
                j -= 1
                state = 'M'
            else:
                align1 = '-' + align1
                align2 = seq2[j-1] + align2
                j -= 1
                state = 'I2'
    
    return final_score, align1, align2

In [19]:
# Последовательности
seq1 = "ATGCAGCAGCAGCCA"
seq2 = "ATATAT"

# Линейный штраф
score_lin, a1_lin, a2_lin = needleman_wunsch_linear(seq1, seq2)
print(f"Score: {score_lin}")
print(f"seq1: {a1_lin}")
print(f"seq2: {a2_lin}")
print()

# Аффинный штраф
score_aff, a1_aff, a2_aff = needleman_wunsch_affine(seq1, seq2)
print(f"Score: {score_aff}")
print(f"seq1: {a1_aff}")
print(f"seq2: {a2_aff}")

Score: -30
seq1: ATGCAGCAGCAGCCA
seq2: AT-----A-TA---T

Score: -15
seq1: ATGCAGCAGCAGCCA
seq2: ATATA---------T


## аффинная модель лучше отражает реальность, потому что в эволюции одно событие вставки/удаления длинного участка ДНК вероятнее, чем множество мелких независимых событий. Высокий штраф за открытие гэпа и низкий за продолжение как раз это моделируют.